# ASR

In [ ]:
!pip install "git+https://github.com/ytdl-org/youtube-dl.git"
!pip install SpeechRecognition
!pip install git+https://github.com/openai/whisper.git soundfile
!pip install assemblyai
!pip install jiwer

## YouTube

Аудио и субтитры скачиваем через youtube-dl.

In [2]:
!youtube-dl --write-auto-sub --sub-lang es -x --audio-format wav https://www.youtube.com/watch?v=BGfRFX90POk

[youtube] BGfRFX90POk: Downloading webpage
[youtube] BGfRFX90POk: Downloading player da154528
[info] Writing video subtitles to: Marta Soto - Entre otros cien (Lyric Video)-BGfRFX90POk.es.vtt
[dashsegments] Total fragments: 1
[download] Destination: Marta Soto - Entre otros cien (Lyric Video)-BGfRFX90POk.webm
[download] 100% of 3.65MiB in 00:00
[ffmpeg] Destination: Marta Soto - Entre otros cien (Lyric Video)-BGfRFX90POk.wav
Deleting original file Marta Soto - Entre otros cien (Lyric Video)-BGfRFX90POk.webm (pass -k to keep)


In [3]:
auto_subs_filename = "/content/Marta Soto - Entre otros cien (Lyric Video)-BGfRFX90POk.es.vtt"
audio_filename = "/content/Marta Soto - Entre otros cien (Lyric Video)-BGfRFX90POk.wav"

Удаляем timestamps, оставляем только субтитры. Видим, что у строк есть дубликаты.

In [4]:
import re

auto_subs = []

with open(auto_subs_filename) as vtt_file:
    for line in vtt_file:
        line = line.strip()
        if len(line) > 0 and line[0].isalpha():
            line = re.sub(r'<.*?>', '', line)
            auto_subs.append(line)

auto_subs[:20]

['WEBVTT',
 'Kind: captions',
 'Language: es',
 'ah',
 'ah',
 'ah',
 'y',
 'y',
 'y',
 'sí',
 'vengo caminando pago un precio razonable',
 'vengo caminando pago un precio razonable',
 'vengo caminando pago un precio razonable',
 'por los actos que me gasto',
 'por los actos que me gasto',
 'por los actos que me gasto',
 'quiero que el mañana me sorprenda sin',
 'quiero que el mañana me sorprenda sin',
 'quiero que el mañana me sorprenda sin',
 'tener una respuesta a preguntas que me']

Удаляем три первые (лишние) строки и схлопываем дубликаты.

In [5]:
auto_subs = auto_subs[3:]
auto_subs = [auto_subs[i] for i in range(len(auto_subs)) if (i==0) or auto_subs[i] != auto_subs[i-1]]
auto_subs = [' '.join(auto_subs)]
print(auto_subs[0][:100])

ah y sí vengo caminando pago un precio razonable por los actos que me gasto quiero que el mañana me 


Функция для препроцессинга (удаляет пунктуацию, приводит к lowercase, делит на токены).

In [6]:
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower().strip()
    text = text.replace("\n", " ")
    return [text]

## Transformers (одна из wav2vec2 моделей для испанского)

In [7]:
from transformers import pipeline

transcriber = pipeline("automatic-speech-recognition", "jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
wav2vec2_subs = transcriber(audio_filename, chunk_length_s=20)["text"]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-spanish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-spanish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
Y

vocab.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

Could not load the `decoder` for jonatasgrosman/wav2vec2-large-xlsr-53-spanish. Defaulting to raw CTC. Error: No module named 'kenlm'
Try to install `kenlm`: `pip install kenlm
Try to install `pyctcdecode`: `pip install pyctcdecode


In [8]:
wav2vec2_subs

'a itdororeimbertopofovengo caminando paga un precio razonable por los sactos que me gasto quiero que el mañana me sorprenda sin tener una respuesta a preguntas que me vengo sin querer venir queriando gamisueños se produzca cuando sigo imaginando querocases movela in tu ifen y saltar a tus botones sin dar es plecación timei mconferesteira odescubro cuando me queda por ever queas lo que pasarán cuando pase otro otri paseando por la calle encontrar rostro entre otros vige mil millones de miradas mes secuestrarán en mil millones de verbúmes pero tú estando leos m pregaras primavecavengo retratando que hoy es siempre y siempre viven el presente de poder ser un granvia quedaron renmentar una manera de soñar y quedarme cualquier viernes dormida en el sofa vengo a terrizando por los miedos cadavía que ahora son abladurías que ye no miempañsar quearó que algún idea pueda berca todavía toda esta palabrarí sigue haciendo debea tremas recones esteiraqun oberscuro cuando me queda pobeda que es lo 

In [9]:
wav2vec2_subs = preprocess(wav2vec2_subs)

## AssemblyAI

In [10]:
import assemblyai as aai

aai.settings.api_key = '057d003bc23c402fbcced728aff1a642'
config = aai.TranscriptionConfig(language_code="es")
transcriber = aai.Transcriber(config=config)
transcript = transcriber.transcribe(audio_filename)

assemblyai_subs = transcript.text
assemblyai_subs

'Vengo caminando, pago un precio razonable por los actos que me gasto Quiero que el mañana me sorprenda sin tener una respuesta a preguntas que me hago Vengo sin querer venir queriendo que mis sueños se produzcan Cuando sigo imaginando Quiero que se esfume la intuición Y saltar a tus botones sin dar explicación Dime si me concedes este amor ¿Cómo descubro cuánto me queda por ver? ¿Qué es lo que pasará cuando pase otro tren? Si paseando por la calle encontraré Tu rostro entre otros diez Cien mil millones de miradas me secuestrarán Cien mil millones de perfumes Pero tú, estando lejos, me regalas primavera Retratando que hoy es siempre y siempre vive en el presente De poder ser un gran día Quiero reinventar una manera de soñar Y quedarme cualquier viernes dormida en el sofá Vengo aterrizando por los miedos que tenía Que ahora son habladurías que ya no me hacen pensar Quiero que algún día pueda ver que todavía Toda esta palabrería Siga siendo de verdad Dime si me concedes este aire ¿Qué? ¿

In [11]:
assemblyai_subs = preprocess(assemblyai_subs)

## Speech Recognition (Whisper)

In [12]:
import speech_recognition as sr

r = sr.Recognizer()
with sr.AudioFile(audio_filename) as source:
    audio = r.record(source)

whisper_subs = r.recognize_whisper(audio, language="spanish")
whisper_subs

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 118MiB/s]


" Vengo caminando pa' con precio razonable por los actos Que me gasto, quiero que el mañana me sorprenda sin tener una respuesta a preguntas que me hago Vengo sin querer venir queriendo que a mis sueños se produzca cuando sigo imaginando, quiero que asesfube la intuición y saltara tus botones sin dar explicación dime si me confes este aire Que como descubro cuanto me queda por ver que es lo que pasará cuando pase otro tren si paseando por la calle encontraré tu brostro entre otros bien y en mil millones de miradas me secuestrarán y en mil millones de perfumes pero tú estándole los negras primaveras Vengo retratando que hoy siempre y siempre viven el presente de poder ser un gran día quiero reingentar una manera de soñar y quedarme cualquier viernes dormida en el sofá Vengo a terrizando por los miedos que tenía que ahora son nebladurías que lleno me hacen pa' yzar quiero que algún día pueda ver que todavía toda esta palabra iría siga siendo de veridad dime si me confes este aire Que com

In [13]:
whisper_subs = preprocess(whisper_subs)

## Эталон

In [14]:
ground_truth = \
"""
Vengo caminando, pago un precio razonable
Por los actos que me gasto
Quiero que el mañana me sorprenda
Sin tener una respuesta
A preguntas que me hago
Vengo, sin querer venir queriendo
Que mis sueños se produzcan
Cuando sigo imaginando
Quiero que se esfume la intuición
Y saltar a tus botones, sin dar explicación
Dime si me concedes este aire
¿Qué?, ¿cómo descubro cuánto me queda por ver?
¿Qué es lo que pasará cuando pase otro tren?
Si paseando por la calle encontraré tu rostro
Entre otros 100, 100 mil millones de miradas me secuestrarán
100 mil millones de perfumes, pero tú
Estando lejos, me regalas primaveras
Vengo retratando que hoy es siempre
Y siempre vive en el presente
De poder ser un gran día
Quiero reinventar una manera de soñar
Y quedarme cualquier viernes, dormida en el sofá
Vengo aterrizando por los miedos que tenía
Que ahora son habladurías, que ya no me hacen pensar
Quiero que, algún día, pueda ver que todavía
Toda esta palabrería siga siendo de verdad
Dime si me concedes este aire
¿Qué?, ¿cómo descubro cuánto me queda por ver?
¿Qué es lo que pasará cuando pase otro tren?
Si paseando por la calle encontraré tu rostro
Entre otros 100, 100 mil millones de miradas me secuestrarán
100 mil millones de perfumes, pero tú
Estando lejos, me regalas primaveras
¿Qué?, ¿cómo descubro cuánto me queda por ver?
¿Qué es lo que pasará cuando pase otro tren?
Si paseando por la calle encontraré tu rostro
Entre otros 100, 100 mil millones de miradas me secuestrarán
100 mil millones de perfumes, pero tú
Estando lejos, me regalas primaveras
"""

In [15]:
ground_truth = preprocess(ground_truth)
print(ground_truth[0][:100])

vengo caminando pago un precio razonable por los actos que me gasto quiero que el mañana me sorprend


## Подсчет WER

In [16]:
import jiwer

print(f"YouTube: {jiwer.wer(ground_truth, auto_subs):.2f}")
print(f"wav2vec2: {jiwer.wer(ground_truth, wav2vec2_subs):.2f}")
print(f"AssemblyAI: {jiwer.wer(ground_truth, assemblyai_subs):.2f}")
print(f"Whisper: {jiwer.wer(ground_truth, whisper_subs):.2f}")

YouTube: 0.27
wav2vec2: 0.52
AssemblyAI: 0.08
Whisper: 0.57


## Итоги

- У Whisper и wav2vec2-модель для испанского результаты оказались почти одинаковыми (wav2vec2 незначительно лучше)
- YouTube-автогенерация лучше, чем Whisper и wav2vec2, но WER все еще высокий
- У AssemblyAI практически эталонный результат